In [1]:
import graph_algo
%load_ext autoreload
%autoreload 2
import copy
from graph_algo.adt import LinkedGraph as LG, MatrixGraph as MG, Vertex, Edge, MapGraph
import  math
from data_structures import LinkedQueue, LinkedStack, SortedPriorityQueue
from graph_algo.union_find import UF


In [4]:
def make_graph(string, max_vertices, directed=False, split='-'):
    g = MapGraph(directed=directed)
    vertices = [Vertex(i) for i in range(max_vertices + 1)]
    for uv in string.split(" "):
        u, v, w = map(lambda x: int(x[1]) if x[0]<2 else float(x[1]),
                   enumerate(uv.split(split)))
        g.insert_edge(Edge(vertices[u], vertices[v], w))
    return g

In [14]:
def dijk_spt(g, s):
    MAX_VAL = float("inf")
    distance = dict()
    path = dict()
    for edge in g.edges:
        edge.is_traversed = False

    for vertex in g.vertices:
        vertex.is_visited = False
        distance[vertex] = MAX_VAL
        path[vertex] = None
    s.is_visited = True
    distance[s] = 0
    path[s] = s
    s.parent = s
    q = LinkedQueue()
    q.enqueue(s)
    while not q.is_empty():
        vertex = q.dequeue()
        for adj in g.adjacent(vertex):
            edge = g.get_edge(vertex, adj)
            if not edge.is_traversed:
                if distance[vertex] < MAX_VAL and\
                        distance[vertex] + edge.weight < distance[adj]:
                    distance[adj] = distance[vertex] + edge.weight
                    path[adj] = vertex
                    adj.parent = vertex
                q.enqueue(adj)
                edge.is_traversed = True

    return {"distance": distance,
            "path": path
            }

graph = make_graph("0-1-0.41 1-2-0.51 2-3-0.5 4-3-0.36 3-5-0.38 3-0-0.45 "
                         "0-5-0.29 5-4-0.21 1-4-0.32 4-2-0.32 5-1-0.29", 5, directed=True)

root = list(graph.vertices)[0]
sp = dijk_spt(graph, root)


In [15]:
sp["path"]

{Vertex(value=0): Vertex(value=0),
 Vertex(value=1): Vertex(value=0),
 Vertex(value=2): Vertex(value=4),
 Vertex(value=3): Vertex(value=4),
 Vertex(value=4): Vertex(value=5),
 Vertex(value=5): Vertex(value=0)}

In [16]:
sp["distance"]



{Vertex(value=0): 0,
 Vertex(value=1): 0.41,
 Vertex(value=2): 0.8200000000000001,
 Vertex(value=3): 0.86,
 Vertex(value=4): 0.5,
 Vertex(value=5): 0.29}

In [17]:
for vertex in graph.vertices:
    print(vertex, vertex.parent)

Vertex(value=0) Vertex(value=0)
Vertex(value=1) Vertex(value=0)
Vertex(value=2) Vertex(value=4)
Vertex(value=3) Vertex(value=4)
Vertex(value=4) Vertex(value=5)
Vertex(value=5) Vertex(value=0)


In [36]:

class Distance:

    def __init__(self, value):
        self.value = value

    def __lt__(self, other):
        return self.value < other.value

def dijk_psq_spt(g, s):
    MAX_VAL = float("inf")
    distance = dict()
    path = dict()
    for edge in g.edges:
        edge.is_traversed = False

    for vertex in g.vertices:
        distance[vertex] = MAX_VAL
        path[vertex] = None
        vertex.is_visited = False

        vertex.distance = Distance(MAX_VAL)

    distance[s] = 0
    s.distance.value = 0
    path[s] = s
    q = SortedPriorityQueue()
    q.add(Distance(0), s)
    o = 0
    while not q.is_empty():
        _, vertex = q.remove_min()
        vertex.is_visited = True
        for adj in g.adjacent(vertex):
            edge = g.get_edge(vertex, adj)
            if not q.contains(adj) and not adj.is_visited:
                print(" ----", adj)
                distance[adj] = distance[vertex] + edge.weight
                adj.distance.value = distance[adj]
                adj.parent = vertex
                path[adj] = vertex
                q.add(adj.distance, adj)

            elif distance[vertex] < MAX_VAL and\
                    vertex.distance.value + edge.weight < adj.distance.value:
                distance[adj] = distance[vertex] + edge.weight
                adj.distance.value = distance[adj]
                path[adj] = vertex
                adj.parent = vertex

    return {"distance": distance,
            "path": path
            }

graph = make_graph("0-1-0.41 1-2-0.51 2-3-0.5 4-3-0.36 3-5-0.38 3-0-0.45 "
                         "0-5-0.29 5-4-0.21 1-4-0.32 4-2-0.32 5-1-0.29", 5, directed=True)

root = list(graph.vertices)[1]
sp =  dijk_psq_spt(graph, root)

<__main__.Distance object at 0x7f767e7b10a0> Vertex(value=1)
 ---- Vertex(value=2)
<__main__.Distance object at 0x7f766ca5d370> Vertex(value=2)
 ---- Vertex(value=4)
<__main__.Distance object at 0x7f766ca5d9d0> Vertex(value=4)
 ---- Vertex(value=3)
<__main__.Distance object at 0x7f766ca5d220> Vertex(value=3)
 ---- Vertex(value=0)
<__main__.Distance object at 0x7f766cc00f70> Vertex(value=0)
 ---- Vertex(value=5)
<__main__.Distance object at 0x7f766ca5d6d0> Vertex(value=5)


In [37]:
sp

{'distance': {Vertex(value=0): 1.13,
  Vertex(value=1): 0,
  Vertex(value=2): 0.51,
  Vertex(value=3): 0.6799999999999999,
  Vertex(value=4): 0.32,
  Vertex(value=5): 1.06},
 'path': {Vertex(value=0): Vertex(value=3),
  Vertex(value=1): Vertex(value=1),
  Vertex(value=2): Vertex(value=1),
  Vertex(value=3): Vertex(value=4),
  Vertex(value=4): Vertex(value=1),
  Vertex(value=5): Vertex(value=3)}}

In [34]:
def floyd_marshal_spt(g):
    MAX_VAL = float("inf")
    for vertex in g.vertices:
        vertex.distance = MAX_VAL
    vertices = list(g.vertices)
    spt = MapGraph(directed=True)
    path = MapGraph(directed=True)
    for u in vertices:
        for v in vertices:
            if u == v:
                weight = 0
                waypoint = u
            elif edge := g.get_edge(u, v):
                weight = edge.weight
                waypoint = u
            else:
                weight = MAX_VAL
                waypoint = None
            edge = Edge(u,v, weight)
            spt.insert_edge(edge)
            path.insert_edge(Edge(u, v, waypoint))
    for w in vertices:
        for u in vertices:
            for v in vertices:
                if u == v:
                    continue
                if w != u and w != v:
                    uw, wv, uv = spt.get_edge(u, w),\
                                 spt.get_edge(w, v),\
                                 spt.get_edge(u, v),
                    if uw.weight + wv.weight < uv.weight:
                        uv.weight = uw.weight + wv.weight
                        edge = path.get_edge(u, v)
                        edge.weight = w
    return {"spt": spt, "path": path}

graph = make_graph("0-1-0.41 1-2-0.51 2-3-0.5 4-3-0.36 3-5-0.38 3-0-0.45 "
                         "0-5-0.29 5-4-0.21 1-4-0.32 4-2-0.32 5-1-0.29", 5, directed=True)

fl = floyd_marshal_spt(graph,)

In [35]:
for vertex in fl["spt"].vertices:
    print(vertex)
    for adj in fl["spt"].adjacent(vertex):
        edge = fl["spt"].get_edge(vertex, adj)
        print(edge)
        print(fl["path"].get_edge(vertex, adj).weight)
    print()
    print()
    print()

Vertex(value=0)
Edge(Vertex(value=0), Vertex(value=0), 0)
Vertex(value=0)
Edge(Vertex(value=0), Vertex(value=1), 0.41)
Vertex(value=0)
Edge(Vertex(value=0), Vertex(value=2), 0.8200000000000001)
Vertex(value=5)
Edge(Vertex(value=0), Vertex(value=3), 0.8599999999999999)
Vertex(value=5)
Edge(Vertex(value=0), Vertex(value=4), 0.5)
Vertex(value=5)
Edge(Vertex(value=0), Vertex(value=5), 0.29)
Vertex(value=0)



Vertex(value=1)
Edge(Vertex(value=1), Vertex(value=0), 1.1300000000000001)
Vertex(value=4)
Edge(Vertex(value=1), Vertex(value=1), 0)
Vertex(value=1)
Edge(Vertex(value=1), Vertex(value=2), 0.51)
Vertex(value=1)
Edge(Vertex(value=1), Vertex(value=3), 0.6799999999999999)
Vertex(value=4)
Edge(Vertex(value=1), Vertex(value=4), 0.32)
Vertex(value=1)
Edge(Vertex(value=1), Vertex(value=5), 1.06)
Vertex(value=4)



Vertex(value=2)
Edge(Vertex(value=2), Vertex(value=0), 0.95)
Vertex(value=3)
Edge(Vertex(value=2), Vertex(value=1), 1.17)
Vertex(value=5)
Edge(Vertex(value=2), Vertex(value=2), 0)
V

In [3]:
MAX_VAL = float("inf")
def dag_longest_weighted_path2(g):
    """
    Topological sort
    For each edge in the sort
    dt[v] = min(dt[v], dt[u] + weight[u, v])
    """
    
    def is_source(v):
        if v.indegree == 0:
            return True
        return False
    sources = LinkedQueue()
    for vertex in g.vertices:
        vertex.is_source = False
        vertex.root = vertex
        vertex.dst = MAX_VAL
        if is_source(vertex):
            vertex.dst = 0
            vertex.is_source = True
            sources.enqueue(vertex)
    while not sources.is_empty():
        vertex = sources.dequeue()
        for adj in g.adjacent(vertex):
            adj.indegree -= 1
            edge = g.get_edge(vertex, adj)
            dst = edge.weight + vertex.weight
            if is_source(adj):
                sources.enqueue(adj)
            adj.dst = min(dst, adj.dst)

In [21]:
def bellman_ford(g, source):
    MAX_VAL = float("inf")
    for vertex in g.vertices:
        vertex.dst = MAX_VAL
    source.is_root = True
    source.dst = 0
    source.path = 0
    for edge in g.edges:
        edge.is_traversed = False
    q = LinkedQueue()
    q.enqueue(source)
    while not q.is_empty():
        vertex = q.dequeue()
        for adj in g.adjacent(vertex):
            edge = g.get_edge(vertex, adj)
            if not edge.is_traversed:
                edge.is_traversed = True
                q.enqueue(adj)
                if not hasattr(adj, 'is_root') and  \
                         adj.dst > vertex.dst + edge.weight:
                    adj.dst = vertex.dst + edge.weight
                    adj.path = vertex
    return g

graph = make_graph("0|1|0.41 1|2|0.51 "
                   "2|3|0.5 4|3|0.36 3|5|-0.38 "
                   "3|0|0.45 "
                    "0|5|0.29 5|4|0.21 1|4|0.32 4|2|0.32 5|1|-0.29", 5, directed=True, split='|')
root = list(graph.vertices)[4]

bf = bellman_ford(graph, root)

Vertex(value=2) inf 0.32
Vertex(value=2)
Vertex(value=3) inf 0.36
Vertex(value=3)
Vertex(value=3) 0.36 0.8200000000000001
Vertex(value=0) inf 0.81
Vertex(value=0)
Vertex(value=5) inf -0.020000000000000018
Vertex(value=5)
Vertex(value=1) inf 1.22
Vertex(value=1)
Vertex(value=5) -0.020000000000000018 1.1
Vertex(value=1) 1.22 -0.31
Vertex(value=1)
Vertex(value=4) 0 0.18999999999999997
Vertex(value=2) 0.32 0.2
Vertex(value=2)
Vertex(value=4) 0 0.010000000000000009


In [22]:
for vertex in bf.vertices:
    print(vertex.dst)

0.81
-0.31
0.2
0.36
0
-0.020000000000000018
